In [1]:
# %% [markdown]
"""
# Airbnb SQL Queries Analysis

This notebook demonstrates how to run SQL queries on Airbnb data using Python.

We'll use SQLite and the `sqlite3` package for this demonstration, but the same concepts apply to other databases.
"""

# %% [markdown]
"""
## Setup and Data Loading
First, let's import necessary packages and set up our SQLite database.
"""

# %%
import sqlite3
import pandas as pd
from IPython.display import display

# Create an in-memory SQLite database (or connect to an existing file)
conn = sqlite3.connect(':memory:')  # Use ':memory:' for temp db or 'airbnb.db' for persistent

# %% [markdown]
"""
## Create Tables and Load Sample Data
For this example, we'll create three tables that represent common Airbnb data:
1. `listings` - Property listings
2. `reviews` - Guest reviews
3. `hosts` - Host information
"""

# %%
# Create listings table
conn.execute('''
CREATE TABLE IF NOT EXISTS listings (
    listing_id INTEGER PRIMARY KEY,
    host_id INTEGER,
    neighborhood TEXT,
    room_type TEXT,
    price REAL,
    minimum_nights INTEGER,
    availability_365 INTEGER,
    number_of_reviews INTEGER
);
''')

# Create reviews table
conn.execute('''
CREATE TABLE IF NOT EXISTS reviews (
    review_id INTEGER PRIMARY KEY,
    listing_id INTEGER,
    date TEXT,
    reviewer_name TEXT,
    comments TEXT,
    rating INTEGER,
    FOREIGN KEY (listing_id) REFERENCES listings(listing_id)
);
''')

# Create hosts table
conn.execute('''
CREATE TABLE IF NOT EXISTS hosts (
    host_id INTEGER PRIMARY KEY,
    host_name TEXT,
    host_since TEXT,
    host_response_rate TEXT,
    host_is_superhost INTEGER,
    FOREIGN KEY (host_id) REFERENCES listings(host_id)
);
''')

# %% [markdown]
"""
### Insert Sample Data
Let's insert some sample data to work with.
"""

# %%
# Sample listings data
listings_data = [
    (1, 101, 'Manhattan', 'Entire home/apt', 150.00, 2, 120, 25),
    (2, 102, 'Brooklyn', 'Private room', 85.00, 3, 90, 18),
    (3, 103, 'Queens', 'Private room', 75.00, 2, 200, 32),
    (4, 101, 'Manhattan', 'Private room', 110.00, 1, 180, 42),
    (5, 104, 'Brooklyn', 'Entire home/apt', 200.00, 2, 60, 12)
]

conn.executemany('INSERT INTO listings VALUES (?, ?, ?, ?, ?, ?, ?, ?)', listings_data)

# Sample reviews data
reviews_data = [
    (1, 1, '2023-01-15', 'Alice', 'Great place!', 5),
    (2, 1, '2023-02-20', 'Bob', 'Lovely apartment', 4),
    (3, 2, '2023-01-10', 'Charlie', 'Nice room', 4),
    (4, 3, '2023-03-05', 'Dana', 'Perfect stay', 5),
    (5, 4, '2023-02-28', 'Eve', 'Good value', 3),
    (6, 4, '2023-03-15', 'Frank', 'Excellent host', 5),
    (7, 5, '2023-01-20', 'Grace', 'Beautiful home', 5)
]

conn.executemany('INSERT INTO reviews VALUES (?, ?, ?, ?, ?, ?)', reviews_data)

# Sample hosts data
hosts_data = [
    (101, 'John Smith', '2018-05-10', '100%', 1),
    (102, 'Maria Garcia', '2019-02-15', '90%', 0),
    (103, 'David Lee', '2020-07-22', '95%', 1),
    (104, 'Sarah Johnson', '2017-11-05', '80%', 0)
]

conn.executemany('INSERT INTO hosts VALUES (?, ?, ?, ?, ?)', hosts_data)

# Commit changes
conn.commit()

# %% [markdown]
"""
## SQL Query Examples
Now let's run some SQL queries on our Airbnb data.
"""

# %% [markdown]
"""
### 1. Basic Query: List all Manhattan listings
"""

# %%
query1 = '''
SELECT * FROM listings
WHERE neighborhood = 'Manhattan'
'''

df1 = pd.read_sql(query1, conn)
display(df1)

# %% [markdown]
"""
### 2. Aggregation: Average price by neighborhood
"""

# %%
query2 = '''
SELECT
    neighborhood,
    AVG(price) as avg_price,
    COUNT(*) as listing_count
FROM listings
GROUP BY neighborhood
ORDER BY avg_price DESC
'''

df2 = pd.read_sql(query2, conn)
display(df2)

# %% [markdown]
"""
### 3. Join: Listings with host information
"""

# %%
query3 = '''
SELECT
    l.listing_id,
    l.neighborhood,
    l.room_type,
    l.price,
    h.host_name,
    h.host_since,
    h.host_is_superhost
FROM listings l
JOIN hosts h ON l.host_id = h.host_id
'''

df3 = pd.read_sql(query3, conn)
display(df3)

# %% [markdown]
"""
### 4. Subquery: Listings with above-average prices
"""

# %%
query4 = '''
SELECT
    listing_id,
    neighborhood,
    room_type,
    price
FROM listings
WHERE price > (SELECT AVG(price) FROM listings)
ORDER BY price DESC
'''

df4 = pd.read_sql(query4, conn)
display(df4)

# %% [markdown]
"""
### 5. Reviews analysis: Average rating by listing
"""

# %%
query5 = '''
SELECT
    l.listing_id,
    l.neighborhood,
    l.room_type,
    l.price,
    AVG(r.rating) as avg_rating,
    COUNT(r.review_id) as review_count
FROM listings l
LEFT JOIN reviews r ON l.listing_id = r.listing_id
GROUP BY l.listing_id, l.neighborhood, l.room_type, l.price
HAVING review_count > 0
ORDER BY avg_rating DESC
'''

df5 = pd.read_sql(query5, conn)
display(df5)

# %% [markdown]
"""
### 6. Window Function: Price ranking within neighborhoods
"""

# %%
query6 = '''
SELECT
    listing_id,
    neighborhood,
    room_type,
    price,
    RANK() OVER (PARTITION BY neighborhood ORDER BY price DESC) as price_rank_in_neighborhood
FROM listings
'''

df6 = pd.read_sql(query6, conn)
display(df6)

# %% [markdown]
"""
### 7. Complex Query: Superhost performance analysis
"""

# %%
query7 = '''
SELECT
    h.host_is_superhost,
    COUNT(DISTINCT l.listing_id) as listing_count,
    AVG(l.price) as avg_price,
    AVG(l.number_of_reviews) as avg_reviews,
    AVG(r.rating) as avg_rating
FROM hosts h
JOIN listings l ON h.host_id = l.host_id
LEFT JOIN reviews r ON l.listing_id = r.listing_id
GROUP BY h.host_is_superhost
'''

df7 = pd.read_sql(query7, conn)
display(df7)

# %% [markdown]
"""
## Using Python Variables in SQL Queries
You can safely insert Python variables into your queries using parameters.
"""

# %%
# Example with parameters
min_price = 100
max_price = 200
neighborhood = 'Brooklyn'

query_params = '''
SELECT * FROM listings
WHERE price BETWEEN ? AND ?
AND neighborhood = ?
'''

df_params = pd.read_sql(query_params, conn, params=(min_price, max_price, neighborhood))
display(df_params)

# %% [markdown]
"""
## Conclusion
This notebook demonstrates how to:
1. Set up a SQLite database in Python
2. Create tables and insert data
3. Run various SQL queries on Airbnb data
4. Visualize results with pandas DataFrames

For a real-world application, you would:
1. Connect to your actual Airbnb database
2. Use the complete schema with all relevant tables
3. Create more complex queries based on your analysis needs
"""

# %%
# Close the database connection when done
conn.close()

,listing_id,host_id,neighborhood,room_type,price,minimum_nights,availability_365,number_of_reviews
0,1,101,Manhattan,Entire home/apt,150.0,2,120,25
1,4,101,Manhattan,Private room,110.0,1,180,42


,neighborhood,avg_price,listing_count
0,Brooklyn,142.5,2
1,Manhattan,130.0,2
2,Queens,75.0,1


,listing_id,neighborhood,room_type,price,host_name,host_since,host_is_superhost
0,1,Manhattan,Entire home/apt,150.0,John Smith,2018-05-10,1
1,2,Brooklyn,Private room,85.0,Maria Garcia,2019-02-15,0
2,3,Queens,Private room,75.0,David Lee,2020-07-22,1
3,4,Manhattan,Private room,110.0,John Smith,2018-05-10,1
4,5,Brooklyn,Entire home/apt,200.0,Sarah Johnson,2017-11-05,0


,listing_id,neighborhood,room_type,price
0,5,Brooklyn,Entire home/apt,200.0
1,1,Manhattan,Entire home/apt,150.0


,listing_id,neighborhood,room_type,price,avg_rating,review_count
0,3,Queens,Private room,75.0,5.0,1
1,5,Brooklyn,Entire home/apt,200.0,5.0,1
2,1,Manhattan,Entire home/apt,150.0,4.5,2
3,2,Brooklyn,Private room,85.0,4.0,1
4,4,Manhattan,Private room,110.0,4.0,2


,listing_id,neighborhood,room_type,price,price_rank_in_neighborhood
0,5,Brooklyn,Entire home/apt,200.0,1
1,2,Brooklyn,Private room,85.0,2
2,1,Manhattan,Entire home/apt,150.0,1
3,4,Manhattan,Private room,110.0,2
4,3,Queens,Private room,75.0,1


,host_is_superhost,listing_count,avg_price,avg_reviews,avg_rating
0,0,2,142.5,15.0,4.5
1,1,3,119.0,33.2,4.4


,listing_id,host_id,neighborhood,room_type,price,minimum_nights,availability_365,number_of_reviews
0,5,104,Brooklyn,Entire home/apt,200.0,2,60,12
